In [10]:
import itertools
import json
import lightgbm as lgb
import pandas as pd
import pickle
from datetime import datetime
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder
import warnings
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error

import numpy as np


warnings.filterwarnings("ignore")

In [11]:
df = pd.read_json('datatran_consolidado.json')
df.head()

,data_inversa,dia_semana,horario,uf,municipio,tipo_acidente,condicao_metereologica,latitude,longitude
0,01/01/2020,quarta-feira,05:40:00,PA,SAO FRANCISCO DO PARA,Saida de leito carrocavel,Ceu Claro,"-1,3101929","-47,74456398"
1,01/01/2020,quarta-feira,06:00:00,MG,UBERABA,Colisao transversal,Ceu Claro,"-19,76747537","-47,98725511"
2,01/01/2020,quarta-feira,06:00:00,BA,CANUDOS,Saida de leito carrocavel,Nublado,"-10,32002103","-39,06425211"
3,01/01/2020,quarta-feira,10:08:00,SP,APARECIDA,Colisao traseira,Sol,"-22,85651665","-45,23114328"
4,01/01/2020,quarta-feira,12:10:00,MG,JUATUBA,Saida de leito carrocavel,Ceu Claro,"-19,947864","-44,381226"


In [12]:
df["data"] = pd.to_datetime(df["data_inversa"], format="%d/%m/%Y", errors="coerce")
df["hora"] = pd.to_datetime(df["horario"], format="%H:%M:%S", errors="coerce").dt.hour

# Remover nulos essenciais para a agregação
df.dropna(subset=['data', 'hora', 'tipo_acidente', 'uf', 'municipio'], inplace=True)

# AGREGACAO CORRIGIDA: Contar acidentes para a combinação única de Data, Local, Tipo e Condição
cols_agrupamento = ['data', 'uf', 'municipio', 'tipo_acidente', 'condicao_metereologica', 'dia_semana']
df_agg = df.groupby(cols_agrupamento).agg(
    acidentes=("data", "count"),
    hora_media=('hora', 'mean')
).reset_index()

print(df_agg.head())

        data  uf             municipio                tipo_acidente  \
0 2020-01-01  AL  MATRIZ DE CAMARAGIBE  Colisao com objeto estatico   
1 2020-01-01  AL    UNIAO DOS PALMARES      Atropelamento de Animal   
2 2020-01-01  AP        FERREIRA GOMES      Atropelamento de Animal   
3 2020-01-01  AP          PORTO GRANDE    Saida de leito carrocavel   
4 2020-01-01  BA               CANUDOS    Saida de leito carrocavel   

  condicao_metereologica    dia_semana  acidentes  hora_media  
0              Ceu Claro  quarta-feira          1         6.0  
1                Nublado  quarta-feira          1         2.0  
2              Ceu Claro  quarta-feira          1         0.0  
3                  Chuva  quarta-feira          1        15.0  
4                Nublado  quarta-feira          1         6.0  


In [13]:
df_agg["dia_semana_num"] = df_agg["data"].dt.dayofweek
df_agg["mes"] = df_agg["data"].dt.month
df_agg["ano"] = df_agg["data"].dt.year
df_agg["dia_do_ano"] = df_agg["data"].dt.dayofyear
df_agg["dia_do_mes"] = df_agg["data"].dt.day

print(f"Total de registros agrupados: {len(df_agg)}")

Total de registros agrupados: 371301


In [14]:
cat_features = ['uf', 'municipio', 'tipo_acidente', 'condicao_metereologica']
mappings = {}

# Aplicar LabelEncoder e salvar mapeamento
for col in cat_features:
    le = LabelEncoder()
    df_agg[col] = le.fit_transform(df_agg[col].astype(str))
    mappings[col] = list(le.classes_)
    
# Garantir sincronia do mapeamento para a interface.py
with open('label_encoder_mappings.json', 'w', encoding='utf-8') as f:
    mappings['dia_semana'] = sorted(df['dia_semana'].unique().tolist())
    json.dump(mappings, f, ensure_ascii=False)
print("'label_encoder_mappings.json' atualizado.")

'label_encoder_mappings.json' atualizado.


In [15]:
features = [
    'uf', 'municipio', 'tipo_acidente', 'condicao_metereologica', 
    'hora_media', 'dia_semana_num', 'mes', 'ano', 'dia_do_ano', 'dia_do_mes'
]

X = df_agg[features]
y = df_agg['acidentes']

# Índices das colunas categóricas para o LightGBM (0 a 3)
cat_indices = [0, 1, 2, 3]

In [16]:
#import sklearn, sys
#reg = lgb.LGBMRegressor(random_state=30, verbose=-1, objective='poisson')
#reg.fit(X_train, y_train)

In [17]:
# Seleciona as features e o target
r2_list = []
rmse_list = []
n_loops = 30

for i in range(n_loops):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i*13)
    
    # --- HIPERPARÂMETROS EXTREMOS PARA MAXIMIZAR R² ---
    # Foco: Aprender mais lentamente (LR baixo) e com mais profundidade (N_Estimators alto)
    model = lgb.LGBMRegressor(
        objective='poisson',       
        n_estimators=3000,         # Aumento extremo no número de árvores
        learning_rate=0.003,       # Diminuição drástica para precisão máxima
        num_leaves=80,             # Aumento na complexidade da árvore
        max_depth=15,              # Profundidade alta
        min_child_samples=10,      # Permite explicar padrões de contagem muito pequena
        reg_alpha=0.5,             # Aumento na regularização L1 (Lasso)
        reg_lambda=0.5,            # Aumento na regularização L2 (Ridge)
        subsample=0.8,             # Amostragem para reduzir variância
        colsample_bytree=0.8,      # Amostragem para reduzir variância
        random_state=i,
        verbose=-1,
        n_jobs=-1
    )
    
    model.fit(X_train, y_train, categorical_feature=cat_indices)
    
    y_pred = model.predict(X_test)
    
    r2_list.append(r2_score(y_test, y_pred))
    rmse_list.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    
    print(f"Rodada {i+1}/{n_loops} -> R²: {r2_list[-1]:.4f} | RMSE: {rmse_list[-1]:.4f}")

print("-" * 40)
print(f"MÉDIA FINAL -> R²: {np.mean(r2_list):.4f} (±{np.std(r2_list):.4f})")
print(f"MÉDIA FINAL -> RMSE: {np.mean(rmse_list):.4f}")

Rodada 1/30 -> R²: 0.4096 | RMSE: 0.1282
Rodada 2/30 -> R²: 0.4104 | RMSE: 0.1282
Rodada 3/30 -> R²: 0.4165 | RMSE: 0.1277
Rodada 4/30 -> R²: 0.4016 | RMSE: 0.1333
Rodada 5/30 -> R²: 0.4170 | RMSE: 0.1287
Rodada 6/30 -> R²: 0.4294 | RMSE: 0.1286
Rodada 7/30 -> R²: 0.4227 | RMSE: 0.1278
Rodada 8/30 -> R²: 0.4222 | RMSE: 0.1322
Rodada 9/30 -> R²: 0.4053 | RMSE: 0.1341
Rodada 10/30 -> R²: 0.4153 | RMSE: 0.1334
Rodada 11/30 -> R²: 0.4129 | RMSE: 0.1327
Rodada 12/30 -> R²: 0.4165 | RMSE: 0.1307
Rodada 13/30 -> R²: 0.4186 | RMSE: 0.1317
Rodada 14/30 -> R²: 0.3902 | RMSE: 0.1350
Rodada 15/30 -> R²: 0.4281 | RMSE: 0.1286
Rodada 16/30 -> R²: 0.4104 | RMSE: 0.1313
Rodada 17/30 -> R²: 0.4268 | RMSE: 0.1277
Rodada 18/30 -> R²: 0.4001 | RMSE: 0.1314
Rodada 19/30 -> R²: 0.4026 | RMSE: 0.1307
Rodada 20/30 -> R²: 0.4239 | RMSE: 0.1315
Rodada 21/30 -> R²: 0.4120 | RMSE: 0.1309
Rodada 22/30 -> R²: 0.4024 | RMSE: 0.1321
Rodada 23/30 -> R²: 0.4134 | RMSE: 0.1289
Rodada 24/30 -> R²: 0.4210 | RMSE: 0.1314
R

In [18]:
model_final = lgb.LGBMRegressor(
    objective='poisson',
    n_estimators=5000, # Número máximo de árvores
    learning_rate=0.001, # Taxa de aprendizado mínima para máximo R²
    num_leaves=90,
    max_depth=20, # Profundidade máxima para complexidade
    min_child_samples=10,
    reg_alpha=0.6,
    reg_lambda=0.6,
    random_state=30,
    verbose=-1
)

model_final.fit(X, y, categorical_feature=cat_indices)

with open('preditor.pkl', 'wb') as f:
    pickle.dump(model_final, f)

print(f"Modelo final salvo em 'preditor.pkl'. ✅")

Modelo final salvo em 'preditor.pkl'. ✅
